## Chapter 7: QAOA with pyQuil

In [46]:
#Libraries
import numpy as np
from functools import partial
from pyquil import Program, api
from pyquil.paulis import PauliSum, PauliTerm, exponential_map, sZ
from pyquil.gates import *
from pyquil import Program, get_qc
from pyquil.api import ForestConnection
from pyquil.api import WavefunctionSimulator
from pyquil.api import QVMConnection
from scipy.optimize import minimize
np.set_printoptions(precision=3, suppress=True)
qvm = QVMConnection()

In [47]:
n_qubits = 2

In [48]:
Hm = [PauliTerm("X", i, 1.0) for i in range(n_qubits)]

In [49]:
J = np.array([[0,1],[0,0]]) # weight matrix of the Ising model. Only the coefficient (0,1) is non-zero.

Hc = []
for i in range(n_qubits):
    for j in range(n_qubits):
        Hc.append(PauliTerm("Z", i, -J[i, j]) * PauliTerm("Z", j, 1.0))

In [50]:
exp_Hm = []
exp_Hc = []
for term in Hm:
    exp_Hm.append(exponential_map(term))
for term in Hc:
    exp_Hc.append(exponential_map(term))

In [51]:
n_iter = 10 # number of iterations of the optimization procedure
p = 1
β = np.random.uniform(0, np.pi*2, p)
γ = np.random.uniform(0, np.pi*2, p)

In [52]:
initial_state = Program()
for i in range(n_qubits):
    initial_state += H(i)

In [53]:
def create_circuit(β, γ):
    circuit = Program()
    circuit += initial_state
    for i in range(p):
        for term_exp_Hc in exp_Hc:
            circuit += term_exp_Hc(-β[i])
        for term_exp_Hm in exp_Hm:
            circuit += term_exp_Hm(-γ[i])

    return circuit

In [54]:
def evaluate_circuit(beta_gamma):
    β = beta_gamma[:p]
    γ = beta_gamma[p:]
    circuit = create_circuit(β, γ)
    return qvm.pauli_expectation(circuit, sum(Hc))

In [55]:
result = minimize(evaluate_circuit, np.concatenate([β, γ]), method='L-BFGS-B')
result

      fun: (-0.9999999999999868+0j)
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0., -0.])
  message: 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 30
      nit: 8
     njev: 10
   status: 0
  success: True
        x: array([0.785, 4.32 ])

#### Explanation

In [56]:
circuit = create_circuit(result['x'][:p], result['x'][p:])

In [57]:
wf_sim = api.WavefunctionSimulator()
state = wf_sim.wavefunction(circuit)
print(state)

(-0.5+0.5j)|00> + (-2.89e-08+4.55e-08j)|01> + (-2.89e-08+4.55e-08j)|10> + (-0.5+0.5j)|11>


In [58]:
print(qvm.pauli_expectation(circuit, PauliSum([sZ(0)])))
print(qvm.pauli_expectation(circuit, PauliSum([sZ(1)])))

0j
0j
